# Imports

In [1]:
import pandas as pd
import numpy as np
from dataclasses import dataclass
import os
import sys
from itertools import cycle

import torch
from torch import nn, randn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, random_split

In [2]:
sys.path.append(os.path.dirname(sys.path[0]))

from deeplogs import Logger, Bar, Reader

# Settings

In [3]:
@dataclass
class Settings():
    
    version: int = 1
    
    epoch: int = 1
    batch_size: int = 16
    lr: float = 1e-3
    device: str = "cuda"
    train_valid_split: float = 0.3
    sched_step_size: int = 1
    sched_gamma: float = 0.1
    
    x_shape: tuple[int] = (1,28,28)
    y_shape: tuple[int] = (10,)
    
    temp: int = 3
    
S = Settings()

# Data Processing

In [4]:
class DS(Dataset):
    
    def __init__(self):
        super(DS, self).__init__()
        
        self.df = [
            pd.read_csv("../data/fashion_mnist/fashion-mnist_train.csv"),
            pd.read_csv("../data/fashion_mnist/fashion-mnist_test.csv"),
        ]
        self.df = pd.concat(self.df).reset_index(drop=True)
        self.unique_labels = np.sort(self.df.label.unique()).tolist()
        
    def __getitem__(self, index):
        idf = self.df.iloc[index]
        x = torch.from_numpy(idf.drop(index=["label"]).to_numpy()).reshape(1,28,28) / 255
        label = idf.loc["label"]
        y = torch.FloatTensor([0 if uniq != label else 1 for uniq in self.unique_labels])
        return x, y
    
    def __len__(self):
        return len(self.df)

# Model

In [5]:
class Model(nn.Module):
    
    def __init__(self):
        super(Model, self).__init__()
        
        self.convblock = nn.Sequential(
            nn.Conv2d(S.x_shape[0], 16, 3),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.cls = nn.Sequential(
            nn.LazyLinear(256),
            nn.ReLU(),
            nn.Linear(256,S.y_shape[0]),
            nn.Softmax(1)
        )
        
        self(randn((1, *S.x_shape)))
        
    def forward(self, x):
        x = self.convblock(x)
        x = torch.flatten(x, 1)
        x = self.cls(x)
        return x

# Training

In [6]:
ds = DS()
x, y = ds.__getitem__(0)
valid_ds_length = int(len(ds) * S.train_valid_split)
train_ds, valid_ds = random_split(ds, [len(ds) - valid_ds_length, valid_ds_length])
train_dl, valid_dl = DataLoader(train_ds, S.batch_size, True), DataLoader(valid_ds, S.batch_size, True)
loss_fc = torch.nn.BCELoss()

In [7]:
model = Model().to(S.device)
S.model = dict(model._modules)
optim = torch.optim.Adam(model.parameters(), S.lr)
print(f"Total Number of Param: {sum([p.numel() for p in model.parameters()]):_}")

/home/guy/.local/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Total Number of Param: 7_956_202


In [8]:
logger = Logger(f"v{S.version}", "This is a description", hyperparams=S.__dict__)

In [9]:
def train(
    model,
    train_dl,
    valid_dl,
    loss_fc,
    optim,
    device,
    logger: Logger,
    epoch,
):
    size = len(train_dl)
    iter_valid_dl = cycle(valid_dl)
    for batch, (x, y) in enumerate(Bar(logger, f"Epoch {epoch+1}", 100)(train_dl)):
        model.train()
        x, y = x.to(device), y.to(device)

        pred = model(x)
        loss = loss_fc(pred, y)

        loss.backward()
        optim.step()
        optim.zero_grad()

        logs = {}
        logs["loss"] = loss.item()
        logs["acc"] = (pred.argmax(1) == y.argmax(1)).float().mean().item()

        with torch.no_grad():
            model.eval()
            x_valid, y_valid = next(iter_valid_dl)
            x_valid, y_valid = x_valid.to(device), y_valid.to(device)
            pred_valid = model(x_valid)
            logs["valid_loss"] = loss_fc(pred_valid, y_valid).item()
            logs["valid_acc"] = (pred_valid.argmax(1) == y_valid.argmax(1)).float().mean().item()
            
        logs["lr"] = optim.param_groups[0]['lr']

        timestep = epoch + (batch/size)
        logger.scalar(timestep, **logs)

In [10]:
for e in range(S.epoch):
    train(model, train_dl, valid_dl, loss_fc, optim, S.device, logger, e)

# Visualise Results

In [11]:
reader = Reader()

In [12]:
reader.infos()

,description,version,epoch,batch_size,lr,device,train_valid_split,sched_step_size,sched_gamma,x_shape,y_shape,temp,model
name,,,,,,,,,,,,,
v1,This is a description,1,1,16,0.001,cuda,0.3,1,0.1,"(1, 28, 28)","(10,)",3,"{'convblock': [Conv2d(1, 16, kernel_size=(3, 3..."


In [13]:
reader.describe()

loss          acc   valid_loss    valid_acc            lr
v1 count  2967.000000  2967.000000  2967.000000  2967.000000  2.967000e+03
   mean      0.076429     0.847679     0.073658     0.848247  1.000000e-03
   std       0.075805     0.105680     0.047671     0.116551  2.168770e-19
   min       0.002139     0.062500     0.002034     0.000000  1.000000e-03
   25%       0.042247     0.812500     0.041211     0.812500  1.000000e-03
   50%       0.065533     0.875000     0.064773     0.875000  1.000000e-03
   75%       0.094973     0.937500     0.094334     0.937500  1.000000e-03
   90%       0.128162     0.937500     0.126853     0.937500  1.000000e-03
   max       2.151497     1.000000     0.385119     1.000000  1.000000e-03

In [14]:
reader.scalar(smooth_perc=0.99)